In [26]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RZZGate, RXXGate, RXGate
import numpy as np

# Define the desired rotation angle
theta = np.pi

# Create a quantum circuit with 2 qubits (target + ancilla)
qc = QuantumCircuit(2, 1)

# Apply RXX to create an effective interaction
qc.append(RXXGate(np.pi/2), [0, 1])

# Apply RZZ to introduce a controlled Z-phase
qc.append(RZZGate(theta), [0, 1])

# Apply another RXX to cancel unwanted effects
qc.append(RXXGate(-np.pi/2), [0, 1])

# Measure the ancilla (optional, for debugging)
qc.measure(0, 0)


In [36]:
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram

simulator = BasicSimulator()

# Simulate the circuit
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()

# Sort the outcomes by frequency (highest first) and take the top 5.
top5 = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]

# Print the top 5 solutions.
print("Top 5 solutions:")
for bitstr, cnt in top5:
    print(f"{bitstr}: {cnt}")


Top 5 solutions:
00001000: 18
00001001: 17
00010100: 16
00000100: 16
00011000: 15


In [35]:
# Define parameters
gamma = 0.5
beta  = 0.1
n_qubits = 8
trotter_steps = 1000
T = 10

# Define quantum registers (8 data qubits + 8 ancilla qubits)
qr = QuantumRegister(2 * n_qubits, name="q")  # 16 qubits in total
cr = ClassicalRegister(n_qubits, name="cr")   # Classical register for measurement
qc = QuantumCircuit(qr, cr)

# Initialize data qubits in superposition
for i in range(n_qubits):
    qc.h(qr[i])

# Function to replace RZ with the fake RZ using a dedicated ancilla
def fake_rz(qc, theta, target, ancilla):
    """Replaces RZ(theta) with an equivalent using only RZZ and RXX, with a dedicated ancilla qubit."""
    qc.append(RXXGate(np.pi / 2), [target, ancilla])  # Entangle with ancilla
    qc.append(RZZGate(theta), [target, ancilla])      # Apply phase shift
    qc.append(RXXGate(-np.pi / 2), [target, ancilla])  # Disentangle

# Trotterized evolution steps
for step in range(trotter_steps):

    alpha = -2 * gamma * (T / trotter_steps)

    # Use separate ancilla qubits for each fake RZ
    fake_rz(qc, alpha * -2.0, 2, 2 + n_qubits)  
    fake_rz(qc, alpha - 1.0, 3, 3 + n_qubits)
    qc.append(RZZGate(alpha * 2.0), [2, 3])

    fake_rz(qc, alpha * -6.0, 4, 4 + n_qubits)
    fake_rz(qc, alpha * -3.0, 5, 5 + n_qubits)
    qc.append(RZZGate(alpha * 2.0), [4, 5])

    fake_rz(qc, alpha * -6.0, 0, 0 + n_qubits)
    fake_rz(qc, alpha * -3.0, 1, 1 + n_qubits)
    qc.append(RZZGate(alpha * 2.0), [0, 1])

    fake_rz(qc, -6.0, 6, 6 + n_qubits)
    fake_rz(qc, -3.0, 7, 7 + n_qubits)
    qc.append(RZZGate(alpha * 2.0), [6, 7])

    # Apply RX gates (unchanged)
    for i in range(n_qubits):
        qc.append(RXGate(2 * beta), [i])

# Measure only the data qubits
qc.measure(range(n_qubits), range(n_qubits))


In [ ]:
import pennylane as qml
import numpy as np
import pyzx
import matplotlib.pyplot as plt

# Define parameters
gamma = 0.5
beta = 0.1
n_qubits = 8
trotter_steps = 1000
T = 10
shots = 1024  # Match Qiskit's number of runs

# Use "default.qubit" with shot-based sampling
dev = qml.device("default.qubit", wires=2 * n_qubits, shots=shots)

def fake_rz(theta, target, ancilla):
    """Replaces RZ(theta) with an equivalent using only RZZ and RXX, with a dedicated ancilla qubit."""
    qml.IsingXX(np.pi / 2, wires=[target, ancilla])  # Entangle with ancilla
    qml.IsingZZ(theta, wires=[target, ancilla])      # Apply phase shift
    qml.IsingXX(-np.pi / 2, wires=[target, ancilla])  # Disentangle

@qml.transforms.to_zx  # Convert to ZX-graph
@qml.qnode(dev)
def circuit():
    # Initialize data qubits in superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Trotterized evolution steps
    for step in range(trotter_steps):
        alpha = -2 * gamma * (T / trotter_steps)

        # Use separate ancilla qubits for each fake RZ
        fake_rz(alpha * -2.0, 2, 2 + n_qubits)
        fake_rz(alpha - 1.0, 3, 3 + n_qubits)
        qml.IsingZZ(alpha * 2.0, wires=[2, 3])

        fake_rz(alpha * -6.0, 4, 4 + n_qubits)
        fake_rz(alpha * -3.0, 5, 5 + n_qubits)
        qml.IsingZZ(alpha * 2.0, wires=[4, 5])

        fake_rz(alpha * -6.0, 0, 0 + n_qubits)
        fake_rz(alpha * -3.0, 1, 1 + n_qubits)
        qml.IsingZZ(alpha * 2.0, wires=[0, 1])

        fake_rz(-6.0, 6, 6 + n_qubits)
        fake_rz(-3.0, 7, 7 + n_qubits)
        qml.IsingZZ(alpha * 2.0, wires=[6, 7])

        # Apply RX gates
        for i in range(n_qubits):
            qml.RX(2 * beta, wires=i)

    # Measure only the data qubits
    return qml.sample(wires=range(n_qubits))

# Convert the circuit into a ZX graph
zx_graph = circuit()

# Draw the ZX-calculus representation using PyZX
fig = pyzx.draw_matplotlib(zx_graph)

# The following lines ensure the figure is properly displayed
manager = plt.figure().canvas.manager
manager.canvas.figure = fig
fig.set_canvas(manager.canvas)

plt.show()
